In [2]:
%pip install nltk

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [nltk]2/3 [nltk]b]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install certifi


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import os
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from tqdm import tqdm

In [65]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pren/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [70]:
def get_wordnet_pos(treebank_tag):
    """Конвертирует POS-теги из формата Penn Treebank в формат WordNet"""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def segment_sentences(text):
    """
    Обрабатывает случаи с сокращениями Dr., Mr., etc.
    """
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = re.split(
        r'(?<![A-Z][a-z]\.)(?<![A-Z]\.)(?<![A-Z][a-z]{2}\.)(?<=[.!?])\s+(?=[\"\']*[A-Z])', 
        text
    )
    return sentences

In [58]:
def tokenize(text):
    """
    Токенизация текста с обработкой сложных случаев:
    - email: user@example.com
    - телефонные номера
    - эмодзи: :), :(, ;)
    """
    pattern = r"""
        [\w.]+@[\w.]+\.[\w.]+
        |
        (?:\+?\d{1,3}[-.\s]?)?                
        \(?\d{3}\)?[-.\s]?                    
        \d{3}[-.\s]?\d{4}                     
        |
        (?:\+?\d{1,3}[-.\s]?)?               
        \(?\d{3}\)?[-.\s]?                    
        \d{3}[-.\s]?\d{2}[-.\s]?\d{2}         
        |
        [:=;xX][oO\-]?[D\)\]\(\]/\\OpP3] 
        |
        \b\w+(?:['.]\w+)*\b
    """
    
    tokens = re.findall(pattern, text, re.VERBOSE)
    return [t for t in tokens if t.strip()]

In [67]:
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

In [68]:
def process_text(text):
    sentences = segment_sentences(text)
    annotated_sentences = []
    
    for sent in sentences:
        tokens = tokenize(sent)
        pos_tags = nltk.pos_tag(tokens)
        annotated_tokens = []
        
        for token, (word, pos_tag) in zip(tokens, pos_tags):
            stem = stemmer.stem(token) if token.isalpha() else token
            
            if token.isalpha():
                wordnet_pos = get_wordnet_pos(pos_tag)
                lemma = lemmatizer.lemmatize(token.lower(), pos=wordnet_pos)
            else:
                lemma = token
                
            annotated_tokens.append((token, stem, lemma))
        
        annotated_sentences.append(annotated_tokens)
    
    return annotated_sentences

In [43]:
def save_annotations(annotations, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, sent in enumerate(annotations):
            for token, stem, lemma in sent:
                f.write(f"{token}\t{stem}\t{lemma}\n")
            if i < len(annotations) - 1:
                f.write("\n")

In [36]:
train_url = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv"
test_url = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv"

train_df = pd.read_csv(train_url, header=None, names=['class', 'title', 'description'])
test_df = pd.read_csv(test_url, header=None, names=['class', 'title', 'description'])

In [46]:
train_df['text'] = train_df['title'] + " " + train_df['description']
test_df['text'] = test_df['title'] + " " + test_df['description']

In [47]:
class_mapping = {
    1: 'World',
    2: 'Sports',
    3: 'Business',
    4: 'Sci_Tech'
}

In [52]:
base_dir = "assets/annotated-corpus"
for dataset in ['train', 'test']:
    for class_name in class_mapping.values():
        os.makedirs(f"{base_dir}/{dataset}/{class_name}", exist_ok=True)

In [69]:
for df, dataset_type in [(train_df.head(100), 'train'), (test_df.head(100), 'test')]:
    print(f"{dataset_type}:")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        doc_id = f"{idx:06d}"
        class_name = class_mapping[row['class']]
        text = row['text']
        
        annotations = process_text(text)
        
        output_path = f"{base_dir}/{dataset_type}/{class_name}/{doc_id}.tsv"
        save_annotations(annotations, output_path)

train:


100%|██████████| 100/100 [00:00<00:00, 1179.07it/s]


test:


100%|██████████| 100/100 [00:00<00:00, 1285.28it/s]


In [41]:
text1 = "She will lead the team to victory."
text2 = "The pipe was made of lead."

for text in [text1, text2]:
    annotations = process_text(text)
    print(f"\n{text}")
    
    for token, stem, lemma in annotations[0]:
        if token.lower() == "lead":
            pos_tag = nltk.pos_tag([token])[0][1]
            print(f"  token: '{token}', lemma: '{lemma}', pos_tag: {pos_tag}")


She will lead the team to victory.

The pipe was made of lead.


In [72]:
text1 = "She can prove it."
text2 = "I open the cans."

for text in [text1, text2]:
    annotations = process_text(text)
    print(f"\n{text}")
    
    for token, stem, lemma in annotations[0]:
        if token.lower() == "can" or token.lower() == "cans":
            pos_tag = nltk.pos_tag([token])[0][1]
            print(f"  token: '{token}', lemma: '{lemma}', pos_tag: {pos_tag}")


She can prove it.
  token: 'can', lemma: 'can', pos_tag: MD

I open the cans.
  token: 'cans', lemma: 'can', pos_tag: NNS
